# 케라스를 사용해 ResNet-34 구현하기

In [1]:
import tensorflow as tf
from tensorflow import keras

In [6]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides, padding="same", use_bias=False),
                keras.layers.BatchNormalization()
            ]
            
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        
        return self.activation(Z + skip_Z)

**Residual unit** 만들기  
- 먼저 생성자에서 필요한 층을 모두 만듦.
- **call** 메서드에서 입력을 main_layer와 skip_layer에 통과시킨 후 두 출력을 더하여 활성 함수를 적용한다.

---


In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[224, 224, 3],
                             padding="same", use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))

prev_filters = 64
for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
    strides = 1 if filters==prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
    print(prev_filters)

model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

64
64
64
128
128
128
128
256
256
256
256
256
256
512
512
512


---
## 사전학습된 ResNet

In [8]:
pretrained_model = keras.applications.resnet50.ResNet50(weights="imagenet")

102973440/102967424 [==============================] - 5s 0us/step


In [14]:
from pprint import pprint

for layer in pretrained_model.layers:
    print("")
    pprint(layer.get_config())
    print(layer.trainable)


{'batch_input_shape': (None, 224, 224, 3),
 'dtype': 'float32',
 'name': 'input_1',
 'ragged': False,
 'sparse': False}
True

{'data_format': 'channels_last',
 'dtype': 'float32',
 'name': 'conv1_pad',
 'padding': ((3, 3), (3, 3)),
 'trainable': True}
True

{'activation': 'linear',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'dtype': 'float32',
 'filters': 64,
 'groups': 1,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'GlorotUniform',
                        'config': {'seed': None}},
 'kernel_regularizer': None,
 'kernel_size': (7, 7),
 'name': 'conv1_conv',
 'padding': 'valid',
 'strides': (2, 2),
 'trainable': True,
 'use_bias': True}
True

{'axis': ListWrapper([3]),
 'beta_constraint': None,
 'beta_initializer': {'class_name': 'Zeros', 'config': {}},
 'beta_regularizer': None,
 'center': True,
 'dtype':

 'strides': (1, 1),
 'trainable': True,
 'use_bias': True}
True

{'axis': ListWrapper([3]),
 'beta_constraint': None,
 'beta_initializer': {'class_name': 'Zeros', 'config': {}},
 'beta_regularizer': None,
 'center': True,
 'dtype': 'float32',
 'epsilon': 1.001e-05,
 'gamma_constraint': None,
 'gamma_initializer': {'class_name': 'Ones', 'config': {}},
 'gamma_regularizer': None,
 'momentum': 0.99,
 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}},
 'moving_variance_initializer': {'class_name': 'Ones', 'config': {}},
 'name': 'conv4_block6_1_bn',
 'scale': True,
 'trainable': True}
True

{'activation': 'relu',
 'dtype': 'float32',
 'name': 'conv4_block6_1_relu',
 'trainable': True}
True

{'activation': 'linear',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'dtype': 'float32',
 'filters': 256,
 'groups': 1,
 'kernel_